In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

# Define dictionaries for column names and types
PROCEDURE_COLUMNS = {f"PROCEDIMIENTO{i}": str for i in range(1, 31)}
SERVICE_COLUMNS = {
    "SERVICIOINGRESO": 0,
    "SERVICIOTRASLADO1": 1,
    "SERVICIOTRASLADO2": 2,
    "SERVICIOTRASLADO3": 3,
    "SERVICIOTRASLADO4": 4,
    "SERVICIOTRASLADO5": 5,
    "SERVICIOTRASLADO6": 6,
    "SERVICIOTRASLADO7": 7,
    "SERVICIOTRASLADO8": 8,
    "SERVICIOTRASLADO9": 9,
    "SERVICIOALTA": 10,
}
DATE_COLUMNS = {
    "FECHA_INGRESO": 0,
    "FECHATRASLADO1": 1,
    "FECHATRASLADO2": 2,
    "FECHATRASLADO3": 3,
    "FECHATRASLADO4": 4,
    "FECHATRASLADO5": 5,
    "FECHATRASLADO6": 6,
    "FECHATRASLADO7": 7,
    "FECHATRASLADO8": 8,
    "FECHATRASLADO9": 9,
    "FECHAALTA": 10,
}

CAMBIO_CAMAS = {
    "AREA MEDICA ADULTO CUIDADOS BASICOS": "Basicos",
    "AREA MEDICA ADULTO CUIDADOS MEDIOS": "Medias",
    "AREA MEDICO-QUIRURGICO CUIDADOS MEDIOS": "Medias",
    "AREA MÉDICA": "Medias",
    "AREA QUIRÚRGICA": "Medias",
    "CIRUGÍA CARDIOVASCULAR": "Medias",
    "UNIDAD DE CUIDADOS INTENSIVOS (UCI) (INDIFERENCIADO)": "UCI",
    "UNIDAD DE CUIDADOS INTENSIVOS ADULTO": "UCI",
    "UNIDAD DE CUIDADOS INTENSIVOS CARDIOLOGÍA": "UCI",
    "UNIDAD DE RECUPERACIÓN DE PABELLONES (CENTRAL Y CMA)": "AMBULATORIO",
    "UNIDAD DE TRATAMIENTO INTERMEDIO (UTI) (INDIFERENCIADO) ADULTO": "UTI",
    "UNIDAD DE TRATAMIENTO INTERMEDIO CARDIOVASCULAR": "UTI",
    "UNIDAD DE TRATAMIENTO INTERMEDIO CIRUGÍA ADULTO": "UTI",
    "UNIDAD DE TRATAMIENTO INTERMEDIO MEDICINA ADULTO": "UTI",
    "UNIDAD DE TRATAMIENTOS INTERMEDIOS MEDICINA": "UTI",
}

# Define relevant diagnoses
RELEVANT_DIAGNOSES = [
    "I35.0",
    "C34.1",
    "C34.2",
    "C34.3",
    "C78.0",
    "C78.2",
    "D38.1",
    "I71.2",
    "I71.0",
    "I35.1",
    "Q20.1",
    "Q20.2",
    "Q20.3",
    "Q20.4",
    "Q20.5",
    "Q20.6",
    "Q20.8",
    "Q20.9",
    "Q21.0",
    "Q21.1",
    "Q21.2",
    "Q21.3",
    "Q21.4",
    "Q21.8",
    "Q22.0",
    "Q22.1",
    "Q22.2",
    "Q22.3",
    "Q22.4",
    "Q22.5",
    "Q22.8",
    "Q23.0",
    "Q23.1",
    "Q23.3",
    "Q24.0",
    "Q24.1",
    "Q24.4",
    "Q24.5",
    "Q24.6",
    "Q24.8",
    "Q24.9",
    "Q25.0",
    "Q25.1",
    "Q25.3",
    "Q25.4",
    "Q25.5",
    "Q25.6",
    "Q25.7",
    "Q25.8",
    "Q25.9",
    "Q26.4",
    "Q26.8",
    "Q27.2",
    "Q27.3",
    "Q28.8",
    "Q28.9",
    "Q31.1",
    "Q32.0",
    "Q32.1",
    "Q33.0",
    "Q33.1",
    "Q33.2",
    "Q33.4",
    "Q33.8",
    "Q34.1",
    "Q34.8",
    "Q67.8",
    "Q76.5",
    "Q76.6",
    "Q76.7",
    "Q76.8",
    "Q76.9",
    "Q78.0",
    "Q79.0",
    "Q79.8",
    "Q83.9",
    "Q85.0",
    "Q85.8",
    "Q85.9",
    "Q87.4",
    "Q89.3",
    "I36.1",
]

In [19]:
df_torax = pd.read_csv("../data/processed/df_procesada_112103.csv", sep=";")
df_torax["FECHAALTA"] = pd.to_datetime(df_torax["FECHAALTA"].str.strip("T00:00:00.000000"))
df_torax = df_torax.sort_values("FECHAALTA")
df_torax = df_torax.reset_index()

C:\Users\jarojas\AppData\Local\Temp\ipykernel_35368\862468.py:1: DtypeWarning: Columns (85,86,88,90) have mixed types. Specify dtype option on import or set low_memory=False.
  df_torax = pd.read_csv("../data/processed/df_procesada_112103.csv", sep=";")


In [ ]:
# Melt service and date columns
service_long = pd.melt(
    df_torax,
    id_vars=["index", "DIAGNOSTICO1", "ANIO_EGRESO", "IR_29301_SEVERIDAD", "CIP_ENCRIPTADO"],
    value_vars=SERVICE_COLUMNS,
    var_name="service_type",
    value_name="service",
)
date_long = pd.melt(
    df_torax,
    id_vars=["index"],
    value_vars=DATE_COLUMNS,
    var_name="date_type",
    value_name="date",
)

# Merge service and date data
patient_journey = pd.merge(date_long, service_long, how="inner", left_index=True, right_index=True)
patient_journey = patient_journey.drop(columns=["index_y"])
patient_journey = patient_journey.rename(columns={"index_x": "index"})

# Convert service type to categorical
patient_journey["service_type"] = pd.Categorical(
    patient_journey["service_type"], categories=SERVICE_COLUMNS.keys(), ordered=True
)

# Sort and clean data
patient_journey = (
    patient_journey.sort_values(["index", "service_type"]).dropna().reset_index(drop=True)
)
patient_journey["date"] = pd.to_datetime(patient_journey["date"], format="mixed")

# Calculate length of stay
patient_journey["length_of_stay"] = patient_journey.groupby("index")["date"].diff().shift(-1)

# Fix negative and zero length of stay
negative_indices = patient_journey[patient_journey["length_of_stay"] < pd.Timedelta(0)].index + 1
patient_journey.loc[negative_indices, "date"] += pd.offsets.DateOffset(years=1)
patient_journey["length_of_stay"] = patient_journey.groupby("index")["date"].diff().shift(-1)

zero_indices = patient_journey[patient_journey["length_of_stay"] == pd.Timedelta(0)].index
patient_journey.loc[zero_indices, "length_of_stay"] += pd.Timedelta(days=1)

# Map bed types
patient_journey["service"] = patient_journey["service"].replace(CAMBIO_CAMAS)

# Summarize length of stay by diagnosis, bed type, and year
summary = (
    patient_journey.groupby(["ANIO_EGRESO", "DIAGNOSTICO1", "IR_29301_SEVERIDAD", "service"]).agg(
        dias_estada_totales=("length_of_stay", "sum"),
        n_pacientes=("CIP_ENCRIPTADO", "nunique"),
        n_egresos=("index", "nunique"),
    )
).reset_index()

# Calculates the percentage of occupancy per bed type
summary["porcentaje_ocupacion"] = summary["dias_estada_totales"] / summary.groupby(
    ["ANIO_EGRESO", "DIAGNOSTICO1"]
)["dias_estada_totales"].transform("sum")

# Calculates length of stay per pacient
summary["dias_estada_por_paciente"] = summary["dias_estada_totales"] / summary["n_pacientes"]

# Filter for relevant diagnoses
summary_relevant = summary.query("DIAGNOSTICO1.isin(@RELEVANT_DIAGNOSES)")

In [21]:
# Obtiene el resumen de dias de estada totales por diagnostico
resumen_dias_de_estada = (
    patient_journey.groupby(["ANIO_EGRESO", "DIAGNOSTICO1", "service"])["length_of_stay"]
    .sum()
    .unstack(fill_value=pd.Timedelta(0))
)

# Total dias de estada por anio y diag
total_dias_de_estada = resumen_dias_de_estada.sum(axis=1)

# Agrega el total de dias ocupados al DataFrame original
resumen_dias_de_estada["total_dias"] = total_dias_de_estada

# Obtiene el porcentaje de ocupacion por cama
porcentaje_de_ocupacion = resumen_dias_de_estada.div(total_dias_de_estada, axis=0)
porcentaje_de_ocupacion = (
    porcentaje_de_ocupacion.stack()
    .to_frame("porcentaje_de_ocupacion")
    .query("porcentaje_de_ocupacion > 0")
)

# Convierte los dias totales a formato long
resumen_dias_de_estada = resumen_dias_de_estada.stack().to_frame("dias_estada_totales")

# Filtra todos los tipos de camas que carezcan de dias
resumen_dias_de_estada = resumen_dias_de_estada[
    resumen_dias_de_estada["dias_estada_totales"] > pd.Timedelta(0)
]

# Une el porcentaje de ocupacion
resumen_dias_de_estada = resumen_dias_de_estada.merge(
    porcentaje_de_ocupacion, how="inner", left_index=True, right_index=True
).reset_index()

# Obtiene el resumen en formato wide
resumen_dias_de_estada_wide = (
    resumen_dias_de_estada.set_index(["ANIO_EGRESO", "DIAGNOSTICO1", "service"])
    .unstack()
    .reset_index(level=1)
)

In [22]:
resumen_dias_de_estada_wide_relevantes = resumen_dias_de_estada_wide[
    resumen_dias_de_estada_wide[("DIAGNOSTICO1", "")].isin(RELEVANT_DIAGNOSES)
]

In [23]:
with pd.ExcelWriter("../data/interim/resumen_ocupacion_por_diagnostico_DE.xlsx") as file:
    resumen_dias_de_estada.to_excel(file, sheet_name="long")
    resumen_dias_de_estada_wide.to_excel(file, sheet_name="wide")
    resumen_dias_de_estada_wide_relevantes.to_excel(file, sheet_name="wide_relevantes")